In [6]:
from sklearn.preprocessing import LabelEncoder
import torch
import os
import cv2 as cv

In [2]:
le = LabelEncoder()
le.fit(['apadding', 'bicycle','human', 'motorcycle', 'vehicle'])

LabelEncoder()

In [3]:
model = torch.load('products/object_detection-5/object_detection-54.pth')
model.eval()
model.to(torch.device('cpu'))

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [13]:
video_path = 'data/train/clips/20200710/clip_35_1535.mp4'
cap = cv.VideoCapture(video_path)
output_path = 'outputs/'
if not os.path.exists(output_path):
    os.makedirs(output_path)
counter = 0
while(True):
    ret, frame = cap.read()
    if (ret == True):
        if counter%25 == 0:
            tens_frame = (frame/255).transpose((2, 0, 1))
            tens_frame = torch.tensor(tens_frame).type(torch.float).unsqueeze(0)
            with torch.no_grad():
                prediction = model(tens_frame)

            boxes = prediction[0]['boxes'].numpy()
            labels = prediction[0]['labels'].numpy()
            scores = prediction[0]['scores'].numpy()
            labels = le.inverse_transform(labels)

            for box, label, score in zip(boxes, labels, scores):
                if score > 0.8:
                    x1,y1,x2,y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])

                    cv.putText(frame,label,(x1,y1-4),cv.FONT_HERSHEY_SIMPLEX,0.4,(0,255,0),1,cv.LINE_AA)
                    cv.rectangle(frame,(x1, y1),(x2, y2),(255,0,0),1)
            path = os.path.join(output_path, 'frame' + str(counter) + '.jpg')
            cv.imshow('video', frame)
            counter += 1
            if cv.waitKey(100) & 0xFF == ord('q'):
                cv.destroyAllWindows()
                break
            
        else:
            counter += 1
        
    else: 
        cv.destroyAllWindows()
        break